In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [2]:
from region.image import Image
from utils import *
import numpy
from skimage.draw import polygon_perimeter, polygon
from scipy.interpolate import interp2d, dfitpack

/Users/reginaongowarsito/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/reginaongowarsito/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)


In [77]:
np.set_printoptions(suppress=True)
# numpy.set_printoptions(threshold=numpy.nan)
numpy.set_printoptions(threshold=1000)


# Utils

In [4]:
np.array([[0, 0], [0, 520], [520, 550], [550, 0]])

array([[  0,   0],
       [  0, 520],
       [520, 550],
       [550,   0]])

In [56]:
def getPoints(im, n):
    n = int(n)
    plt.imshow(im.imArr)
    points = plt.ginput(n, timeout = 300, show_clicks=True)
    plt.close()
    print("Points gotten from", im.name, ":", str(points))
    return np.array(points)

def dispPtsOnImg(im, results, tag=False):
    dispPtsOnImgRC(im, results[:, 0], results[:, 1], tag)

def dispPtsOnImgRC(im, rowInds, colInds, tag=False):
    assert len(rowInds) == len(colInds)
    plt.imshow(im)
    plt.plot(rowInds, colInds, 'bo')
    if tag:
        for ind in range(len(rowInds)):
            x, y = rowInds[ind], colInds[ind]
            plt.text(x, y, str(ind), bbox=dict(facecolor='blue', alpha=0.5))
            plt.text(x, y, "%d-%d" % (x,y), bbox=dict(facecolor='red', alpha=0.5))
    plt.waitforbuttonpress()
    plt.close()

In [ ]:
imT1 = Image.fromPath("../sample_imgs/20181103_162858.jpg")

In [ ]:
viewImage(imT1.imArr)

In [6]:
def computeH(startPts, endPts):
    assert len(startPts) == len(endPts)
    A, b = createA(startPts, endPts)
#     Axx = createAb(startPts, endPts)
#     bxx = np.zeros(len(Axx))
#     bxx[-1] = 1
#     print("whoomp: ", numpy.linalg.lstsq(Axx, bxx)[0])
    
    x = numpy.linalg.lstsq(A, b)
    result = np.append(x[0], 1)
    return result.reshape(3, 3)

In [ ]:
computeH(im1Pts, im2Pts)

In [13]:
imSproul0 = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_00.jpg", 0.25)
imSproul0.readCoordsIn("../curated_sample_imgs/lower_sproul/lower_sproul_00.txt")
imSproul0.pts = imSproul0.pts * 0.25
imSproul1 = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_01.jpg", 0.25)
imSproul1.readCoordsIn("../curated_sample_imgs/lower_sproul/lower_sproul_01.txt")
imSproul1.pts = imSproul1.pts * 0.25

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [8]:
def createAb(startPts, endPts):
    result = np.zeros((len(startPts) * 2 + 1, 9))
    for i in range(len(startPts)):
        startPoint = startPts[i]
        endPoint = endPts[i]
        
        # first row set
        result[i*2, 0] = -1 * startPoint[0]
        result[i*2, 1] = -1 * startPoint[1]
        result[i*2, 2] = -1
        
        result[i*2, 6] = startPoint[0] * endPoint[0]
        result[i*2, 7] = startPoint[1] * endPoint[0]
        result[i*2, 8] = endPoint[0]
        
        
        # second row set
        result[i*2+1, 3] = -1 * startPoint[0]
        result[i*2+1, 4] = -1 * startPoint[1]
        result[i*2+1, 5] = -1
        
        result[i*2+1, 6] = startPoint[0] * endPoint[1]
        result[i*2+1, 7] = startPoint[1] * endPoint[1]
        result[i*2+1, 8] = endPoint[1]
    result[-1, -1] = 1
    
    return result


def createA(startPts, endPts):
    result = np.zeros((len(startPts) * 2, 8))
    b = np.zeros(len(result))
    for i in range(len(startPts)):
        startPoint = startPts[i]
        endPoint = endPts[i]
        
        # first row set
        result[i*2, 0] = -1 * startPoint[0]
        result[i*2, 1] = -1 * startPoint[1]
        result[i*2, 2] = -1
        
        result[i*2, 6] = startPoint[0] * endPoint[0]
        result[i*2, 7] = startPoint[1] * endPoint[0]
        b[i*2] = -endPoint[0]
        
        
        # second row set
        result[i*2+1, 3] = -1 * startPoint[0]
        result[i*2+1, 4] = -1 * startPoint[1]
        result[i*2+1, 5] = -1
        
        result[i*2+1, 6] = startPoint[0] * endPoint[1]
        result[i*2+1, 7] = startPoint[1] * endPoint[1]
        b[i*2+1] = -endPoint[1]
    # result[-1, -1] = 1
    
    return result, b
    

# def createA(numOfPts):
#     result = np.zeros((numOfPts * 2, 8))
    
#     for i in range()

# def createA(numOfPts):
#     result = np.zeros((numOfPts * 2, 8))
    
#     for i in range()

In [ ]:
dispPtsOnImg(imSproul0.imArr, imSproul0.pts)
dispPtsOnImg(imSproul1.imArr, imSproul1.pts)

In [ ]:
"""
p' = H p
where
    p' = target
    p = source

place on p' the color on p

---

get the points in final image that actually will need a value from source image
    do by calculating the results of the corners of the source img getting transformed,
    then using polygon to get the coordinates that actually lies inside the polygon, so you dont do the coords that isnt in it
    

for every point in p', get the corresponding p with p' * H^-1

"""

In [ ]:
sourcePts = imSproul0.pts
targetPts = imSproul1.pts

H = computeH(sourcePts, targetPts)
print("source:", sourcePts)
print("transformed source:", transf(H, sourcePts))
print("target:", targetPts)

dispPtsOnImg(imSproul0.imArr, sourcePts)
dispPtsOnImg(imSproul1.imArr, transf(H, sourcePts))

In [9]:
def transf(H, res, normalize=True):
    if res.ndim == 1:
        vect = np.append(res, 1)
    else:
        vect = np.vstack((res.T, np.ones(len(res))))
    tfSourceCorners = np.dot(H, vect)
    
    if normalize:
        for i in range(tfSourceCorners.shape[1]):
            corner = tfSourceCorners[:, i]
            normCorner = np.dot(corner, 1/(corner[2]))
            tfSourceCorners[:, i] = normCorner
    return tfSourceCorners[:-1, :].T

In [10]:
def getDimensions(imCornersCoords):
    # assuming imCornersCoords = [[width, height]]
    
    minWidth = int(np.floor(min(imCornersCoords[:, 0])))
    minHeight = int(np.floor(min(imCornersCoords[:, 1])))
    
    maxWidth = int(np.ceil(max(imCornersCoords[:, 0])))
    maxHeight = int(np.ceil(max(imCornersCoords[:, 1])))
    
    totalWidth = maxWidth - minWidth + 1
    totalHeight = maxHeight - minHeight + 1
    
    tup = totalHeight, totalWidth, minHeight, minWidth
    print(tup)
    
    return tup
    

In [112]:
def findAllCoords(tfSourceCorners):
    
    xCorners = tfSourceCorners[:, 0]
    yCorners = tfSourceCorners[:, 1]
    
    flatMinX = int(np.floor(min(xCorners)))
    flatMinY = int(np.floor(min(yCorners)))
    newXCorners = xCorners - flatMinX
    newYCorners = yCorners - flatMinY
    
    
    innerCoordsR, innerCoordsC = polygon(newXCorners, newYCorners)
    finInnerCoordsR, finInnerCoordsC = innerCoordsR + flatMinX, innerCoordsC + flatMinY
    
    
    perimCoordsR, perimCoordsC = polygon_perimeter(xCorners, yCorners)
    allCoordsR = np.append(finInnerCoordsR, perimCoordsR)
    allCoordsC = np.append(finInnerCoordsC, perimCoordsC)

#     allCoordsR = finInnerCoordsR
#     allCoordsC = finInnerCoordsC

#     allCoordsR = perimCoordsR
#     allCoordsC = perimCoordsC

    targetPs = np.vstack((allCoordsR, allCoordsC)).T
    
    return targetPs

In [107]:
class WarpedImageInfo:
    
    def __init__(self, corners, targetPs, rvals, gvals, bvals):
        self.corners = corners
        self.targetPs = targetPs
        self.rvals = rvals
        self.gvals = gvals
        self.bvals = bvals
        
    def createImageObj(self, newImage=None, minH = 0, minW = 0, name = ""):
        if newImage == None:
            finH, finW, minH, minW = getDimensions(self.corners)
            newImage = Image(name, np.zeros((finH, finW, 3)))
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 0] = self.rvals
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 1] = self.gvals
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 2] = self.bvals
        return newImage

def warpTo(imSource, sourcePts, targetPts):

    H = computeH(sourcePts, targetPts)
    sourceCorners = Image.getCorners(imSource)
    tfSourceCorners = transf(H, sourceCorners)

    print("sourceCorners:", sourceCorners)
    print("tfSourceCorners:", tfSourceCorners)

    targetPs = findAllCoords(tfSourceCorners)

    invH = np.linalg.inv(H)

    sourcePs = transf(invH, targetPs)
    
    source_Rchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 0])
    source_Gchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 1])
    source_Bchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 2])
    autoInterp = lambda f, xs, ys: dfitpack.bispeu(f.tck[0], f.tck[1], f.tck[2], f.tck[3], f.tck[4], xs, ys)[0]

    Rvals = autoInterp(source_Rchan, sourcePs[:, 0], sourcePs[:, 1])
    Gvals = autoInterp(source_Gchan, sourcePs[:, 0], sourcePs[:, 1])
    Bvals = autoInterp(source_Bchan, sourcePs[:, 0], sourcePs[:, 1])
    
    
    return WarpedImageInfo(tfSourceCorners, targetPs, Rvals, Gvals, Bvals)

def warpProcess(imBase, imBasePts, imgList):

    warpedResults = []
    for im in imgList:

        imPts = im.pts

        warpedResults.append(warpTo(im, imPts, imBasePts))

    allCorners = np.vstack([warped.corners for warped in warpedResults])
    print(allCorners)
    finH, finW, minH, minW = getDimensions(allCorners)
    # finalImage = Image("finalImage", np.zeros((finH, finW, 3)))

    newWarpedImages = []
    # for warped in warpedResults:
    for i in range(len(warpedResults)):
        warped = warpedResults[i]
        newImage = Image("finalImage", np.zeros((finH, finW, 3)))
        newWarpedImages.append(warped.createImageObj(newImage, minH = minH, minW = minW))
    
    return newWarpedImages

In [ ]:
imgList = [imSproul0, imSproul1]
imBase = imSproul1
imBasePts = imSproul1.pts

result = warpProcess(imBase, imBasePts, imgList)
for i in range(len(result)):
    im = result[i]
    printImage(str(i) + ".jpg", im.imArr)

In [60]:
imRect = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_00.jpg", 0.25)
imRect.readCoordsIn("../curated_sample_imgs/lower_sproul/rect_orig_lower_sproul_00.txt")
imRect.pts = imRect.pts * 0.25
dispPtsOnImg(imRect.imArr, imRect.pts)

targPts = Image.readCoords("../curated_sample_imgs/lower_sproul/rect_targ_lower_sproul_00.txt")
targPts = targPts * 0.25
print(targPts)
dispPtsOnImg(imRect.imArr, targPts)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[  0.    0. ]
 [  0.  130. ]
 [130.  137.5]
 [137.5   0. ]]


In [113]:
res = warpTo(imRect, imRect.pts, targPts)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if __name__ == '__main__':


sourceCorners: [[   0    0]
 [   0  489]
 [1007  489]
 [1007    0]]
tfSourceCorners: [[-1851.51890123  -384.08664719]
 [-1275.92323581   549.85174944]
 [  268.97870013   267.84120638]
 [  281.92799456  -149.90243403]]


In [114]:
res.targetPs

array([[-1851,  -384],
       [-1850,  -383],
       [-1850,  -382],
       ...,
       [-1850,  -384],
       [-1851,  -384],
       [-1852,  -384]])

In [118]:
im = res.createImageObj()
printImage("rectified.jpg", im.imArr)

(936, 2135, -385, -1852)
File saved to: output_imgs/20181110180317_rectified.jpg


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [94]:
polygon([0, 0.5, 0.5, 0], [0, 0, 0.5, 0.5])

(array([0]), array([0]))

In [ ]:
imTestImage = Image.fromPath("../curated_sample_imgs/test_image.png")
imSource = imTestImage

source_Rchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 0])
source_Gchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 1])
source_Bchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 2])

autoInterp = lambda f, xs, ys: dfitpack.bispeu(f.tck[0], f.tck[1], f.tck[2], f.tck[3], f.tck[4], xs, ys)[0]

######

sourceCorners = np.array([[0, 0], [0, 479], [479, 479], [479, 0]])

tfSourceCorners = np.array([[99, 99], [99, 579], [579, 579], [579, 99]])

H = computeH(sourceCorners, tfSourceCorners)
print(H)


###


innerCoordsR, innerCoordsC = polygon(tfSourceCorners[:, 0], tfSourceCorners[:, 1])
perimCoordsR, perimCoordsC = polygon_perimeter(tfSourceCorners[:, 0], tfSourceCorners[:, 1])
allCoordsR = np.append(innerCoordsR, perimCoordsR)
allCoordsC = np.append(innerCoordsC, perimCoordsC)

targetPs = np.vstack((allCoordsC, allCoordsR)).T


invH = np.linalg.inv(H)

sourcePs = transf(invH, targetPs)


finH = int(np.ceil(max(targetPs[:, 1])))
finW = int(np.ceil(max(targetPs[:, 0])))
finalImage = Image("finalImage", np.zeros((finH + 1, finW + 1, 3)))
print(finalImage.imArr.shape)

Rvals = autoInterp(source_Rchan, sourcePs[:, 0], sourcePs[:, 1])
Gvals = autoInterp(source_Gchan, sourcePs[:, 0], sourcePs[:, 1])
Bvals = autoInterp(source_Bchan, sourcePs[:, 0], sourcePs[:, 1])
print(targetPs)

finalImage.imArr[targetPs[:, 1], targetPs[:, 0], 0] = Rvals
finalImage.imArr[targetPs[:, 1], targetPs[:, 0], 1] = Gvals
finalImage.imArr[targetPs[:, 1], targetPs[:, 0], 2] = Bvals

viewImage(finalImage.imArr)

In [ ]:
from skimage.draw import polygon_perimeter
img = np.zeros((10, 10), dtype=np.uint8)
rr, cc = polygon_perimeter([6, 1, 5, 10],
                           [1, 5, 11, 5],
                           shape=img.shape, clip=True)
img[rr, cc] = 1
img

In [ ]:
np.append(np.array([1, 2, 3]), 1)

In [ ]:
a = np.array([[0, 0]])
b = np.array([[2, 2]])
print(createA(a, b))
print(createAb(a, b))